# Biol 359A | Mastering linear algebra
### Spring 2025, Week 8
### Learning Objectives:
- Construct and manipulate vectors and matrices (addition, multiplication, transpose, and inverse calculations)
- Apply and matrix transformations (rotation, reflection, scaling) and visualize how these transformations affect shapes in 2D space
- Calculate eigenvalues and eigenvectors of matrices
- Analyze the stability of linear differential equation systems by analyzing eigenvalue:
  - **All negative:** Stable (Converging to equilibrium)  
  - **All positive:** Unstable (Diverging from equilibrium)  
  - **Complex with imaginary components:** Oscillatory (Periodic motion)  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as la
from scipy.integrate import odeint
import seaborn as sns
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## Introduce vector and matrics

### Vectors and matrics representation in Python

In [ ]:
print("REPRESENTING VECTORS AND MATRICES")
print("-" * 40)

# Creating vectors
vector_2d = np.array([3, 4])
vector_3d = np.array([1, 2, 3])

print("2D Vector:", vector_2d)
print("3D Vector:", vector_3d)
print(f"Vector magnitude (2D) of {vector_2d}:", np.linalg.norm(vector_2d))

# Creating matrices
matrix_2x2 = np.array([[1, 2], 
                       [3, 4]])

matrix_3x3 = np.array([[1, 0, 2], 
                       [0, 1, 3], 
                       [2, 1, 0]])

print("\n2x2 Matrix:")
print(matrix_2x2)
print("\n3x3 Matrix:")
print(matrix_3x3)

### Visualize vectors

In [ ]:
def plot_2d_vectors(vectors, labels=None, colors=None):
    """Plot 2D vectors starting from origin"""
    fig, ax = plt.subplots(figsize=(6, 6))
    
    if colors is None:
        colors = ['red', 'blue', 'green', 'orange', 'purple']
    
    for i, v in enumerate(vectors):
        color = colors[i % len(colors)]
        ax.quiver(0, 0, v[0], v[1], angles='xy', scale_units='xy', scale=1, 
                 color=color, width=0.005, alpha=0.8)
        
        # Add labels
        if labels:
            ax.text(v[0] + 0.1, v[1] + 0.1, labels[i], fontsize=12, color=color)
    
    ax.set_xlim(-1, 6)
    ax.set_ylim(-1, 6)
    ax.grid(True, alpha=0.3)
    ax.set_aspect('equal')
    ax.set_title('2D Vectors', fontsize=14, fontweight='bold')
    ax.set_xlabel('X-axis')
    ax.set_ylabel('Y-axis')
    plt.show()
vectors_to_plot = [vector_2d, np.array([2, 1]), np.array([1, 3])]
labels = ['v1 = [3,4]', 'v2 = [2,1]', 'v3 = [1,3]']
plot_2d_vectors(vectors_to_plot, labels)

## Matrix transformations

### Rotation

In [ ]:
# Original points forming a simple shape (square)
square_points = np.array([[0, 1, 1, 0, 0],   # x-coordinates
                          [0, 0, 1, 1, 0]])   # y-coordinates

def apply_transformation(matrix, points):
    """Apply transformation matrix to points"""
    return matrix @ points # We use "@" in Python to do matrix multiplication operation

def plot_transformation(original, transformed, title):
    """Plot original and transformed shapes"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Original shape
    ax1.plot(original[0], original[1], 'bo-', linewidth=2, markersize=8)
    ax1.set_title('Original Shape', fontweight='bold')
    ax1.grid(True, alpha=0.3)
    ax1.set_aspect('equal')
    ax1.set_xlim(-2, 3)
    ax1.set_ylim(-2, 3)
    
    # Transformed shape
    ax2.plot(transformed[0], transformed[1], 'ro-', linewidth=2, markersize=8, label="transformed points")
    ax2.plot(original[0], original[1], 'bo-', alpha=0.3, linewidth=1, label="original points")
    ax2.set_title(f'After {title}', fontweight='bold')
    ax2.grid(True, alpha=0.3)
    ax2.set_aspect('equal')
    ax2.set_xlim(-2, 3)
    ax2.set_ylim(-2, 3)
    ax2.legend()
    plt.tight_layout()
    plt.show()

# Rotation matrix (45 degrees counterclockwise)
angle = np.pi / 4  # 45 degrees in radians
rotation_matrix = np.array([[np.cos(angle), -np.sin(angle)],
                           [np.sin(angle),  np.cos(angle)]])

print("Rotation Matrix (45° CCW):")
print(rotation_matrix)

rotated_square = apply_transformation(rotation_matrix, square_points)
plot_transformation(square_points, rotated_square, 'Rotation (45°)')

### Flip

In [ ]:
# Reflection matrix (flip across y-axis)
reflection_matrix = np.array([[-1, 0],
                             [ 0, 1]])

print("\nReflection Matrix (across y-axis):")
print(reflection_matrix)

reflected_square = apply_transformation(reflection_matrix, square_points)
plot_transformation(square_points, reflected_square, 'Reflection (y-axis)')

### Scaling

In [ ]:
# Scaling matrix
scaling_matrix = np.array([[2, 0],
                          [0, 0.5]])

print("\nScaling Matrix (2x in x, 0.5x in y):")
print(scaling_matrix)

scaled_square = apply_transformation(scaling_matrix, square_points)
plot_transformation(square_points, scaled_square, 'Scaling')

## Eigenvalues and eigenvectors

### Example calculations

In [ ]:
# Example matrix
M = np.array([[3, 1], [0, 2]])
print("Matrix M:")
print(M)

# Calculate eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eig(M)

print(f"\nEigenvalues: {eigenvalues}")
print(f"Eigenvectors:\n{eigenvectors}\n")

# Verification: M × v = λ × v
print("=" * 10, "Validate the eigenvalue/eigenvector", "=" * 10)
for i in range(len(eigenvalues)):
    v = eigenvectors[:, i]
    lambda_val = eigenvalues[i]
    
    left_side = M @ v
    right_side = lambda_val * v
    
    print(f"\nFor eigenvalue λ{i+1} = {lambda_val:.3f}:")
    print(f"M × v{i+1} = {left_side}")
    print(f"λ{i+1} × v{i+1} = {right_side}")
    print(f"Equal? {np.allclose(left_side, right_side)}")

### Visualize eigenvectors

In [ ]:
# Visualizing eigenvectors
def plot_eigenvectors(matrix, eigenvals, eigenvecs):
    """Visualize eigenvectors and their transformations"""
    fig, ax = plt.subplots(figsize=(6, 5))
    for i in range(len(eigenvals)):
        v = eigenvecs[:, i]
        lambda_val = eigenvals[i]
        # Transformed eigenvector (M × v = λ × v)
        transformed = matrix @ v
        ax.quiver(0, 0, transformed[0], transformed[1], angles='xy',
                 scale_units='xy', scale=1, color=f'C{i}', width=0.008,
                 label=f'M×v{i+1} = {lambda_val:.2f}×v{i+1}')
        # Original eigenvector
        ax.quiver(0, 0, v[0], v[1], angles='xy', scale_units='xy', scale=1,
                 color='k', width=0.005, alpha=0.7,
                 label=f'v{i+1} (eigenvector)')
        # Add labels to the eigenvectors
        ax.text(v[0]/2, v[1]/2, f'v{i+1}', fontsize=12, color='k', ha='center', va='center')

    
    ax.set_xlim(-4, 4)
    ax.set_ylim(-4, 4)
    ax.grid(True, alpha=0.3)
    ax.set_aspect('equal')
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax.set_title('Eigenvectors: Special directions preserved by transformation',
                fontweight='bold')
    plt.tight_layout()
    plt.show()

plot_eigenvectors(M, eigenvalues, eigenvectors)

## System of linear differential equations: dx/dt = A*x

In [ ]:
def solve_linear_system(A, x0, t_span, title):
    """Solve dx/dt = A*x and plot the solution"""
    
    def system(x, t):
        return A @ x
    
    t = np.linspace(0, t_span, 1000)
    sol = odeint(system, x0, t)
    eigenvals = np.linalg.eig(A)[0]
    print(f"Eigenvalues: {eigenvals}")
    
    # Create plots
    fig, ax = plt.subplots(1, 1, figsize=(8, 4))
    
    # Time series plot
    ax.plot(t, sol[:, 0], label='$x_1(t)$', linewidth=2)
    ax.plot(t, sol[:, 1], label='$x_2(t)$', linewidth=2)
    ax.set_xlabel('Time')
    ax.set_ylabel('State variables')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    eigenvals_str = np.array2string(eigenvals, precision=2, separator=', ')
    plt.suptitle(f'Linear System: dx/dt = A·x\nEigenvalues: {eigenvals_str}', 
                    fontsize=14, fontweight='bold')
    plt.show()
    
    return eigenvals
x0 = np.array([2, 1])

### Stable

In [ ]:
A = np.array([[-2, -1],
              [-1, -3]])
print("Stable Node (Sink), e.g., cooling of a hot object in a room following Newton's Law of Cooling.")
print("Matrix A:")
print(A)
eigenvals1 = solve_linear_system(A, x0, 3, "Unstable Node")

### Unstable

In [ ]:
A = np.array([[1, 0.5], 
               [0, 2]])

print("UNSTABLE NODE (exponential growth)")
print("Matrix A:")
print(A)
eigenvals2 = solve_linear_system(A, x0, 3, "Unstable Node")

### Oscillatory

In [ ]:
A = np.array([[-0.5, -2], 
               [ 2, -0.5]])

print("Damped spring-mass system)")
print("Matrix A:")
print(A)
eigenvals3 = solve_linear_system(A, x0, 10, "Damped spring-mass system")

In [ ]:
A = np.array([[0, -1], 
               [1,  0]])

print("CENTER (pure oscillation, like a frictionless pendulum)")
print("Matrix A:")
print(A)
eigenvals4 = solve_linear_system(A, x0, 10, "Center (Pure Oscillation)")

### Reference material - matrices operation in python

In [ ]:
# Matrix operations
A = np.array([[2, 1], [1, 3]])
B = np.array([[1, 2], [0, 1]])

print("Matrix A:")
print(A)
print("\nMatrix B:")
print(B)
print("=" * 50)
# Addition
print("\nA + B =")
print(A + B)

# Multiplication
print("\nA × B =")
print(A @ B)

# Transpose
print("\nA^T (transpose of A) =")
print(A.T)

# Determinant
print(f"\ndet(A) = {np.linalg.det(A):.3f}")
print(f"det(B) = {np.linalg.det(B):.3f}")

# Inverse
try:
    A_inv = np.linalg.inv(A)
    print("\nA^(-1) (inverse of A) =")
    print(A_inv)
    
    # Verify: A × A^(-1) = I
    print("\nVerification A × A^(-1) =")
    print(A @ A_inv)
except np.linalg.LinAlgError:
    print("\nA is not invertible (singular matrix)")

# Matrix rank
print(f"\nRank of A: {np.linalg.matrix_rank(A)}")
print(f"Rank of B: {np.linalg.matrix_rank(B)}")